In [ ]:
import pandas as pd
import re

# Load the dataset (reusing loading logic for robustness)
try:
    df = pd.read_csv('final_food_delivery_dataset.csv')
except:
    # Fallback to regeneration
    orders_df = pd.read_csv('orders.csv')
    users_df = pd.read_json('users.json')
    restaurant_data = []
    with open('restaurants.sql', 'r') as f:
        for line in f:
            if line.strip().startswith("INSERT INTO restaurants VALUES"):
                match = re.search(r"\((.*)\);", line)
                if match:
                    restaurant_data.append([p.strip().strip("'") for p in match.group(1).split(',')])
    restaurants_df = pd.DataFrame(restaurant_data, columns=['restaurant_id', 'restaurant_name', 'cuisine', 'rating'])
    restaurants_df['restaurant_id'] = pd.to_numeric(restaurants_df['restaurant_id'])
    merged = pd.merge(orders_df, users_df, on='user_id', how='left')
    df = pd.merge(merged, restaurants_df, on='restaurant_id', how='left', suffixes=('_order', '_details'))

# Calculate counts
total_orders = len(df)
gold_orders = len(df[df['membership'] == 'Gold'])

# Calculate percentage
percentage = (gold_orders / total_orders) * 100

# Round to nearest integer
rounded_percentage = round(percentage)

print(f"Total Orders: {total_orders}")
print(f"Gold Orders: {gold_orders}")
print(f"Percentage: {percentage}")
print(f"Rounded Percentage: {rounded_percentage}")